In [ ]:
import magenta
import note_seq
import tensorflow
from note_seq.protobuf import music_pb2

# set starting pitch note on piano keyboard, you can adjust as per raga needs
pitch = 60 # C5
# 12 notes on piano keyboard
sa,ri1,ri2, ga1,ga2, ma1, ma2, pa, da1,da2, ni1,ni2, sam = pitch, pitch+1, pitch+2, pitch+3, pitch+4, pitch+5,pitch+6,pitch+7,pitch+8,pitch+9,pitch+10,pitch+11,pitch+12
# mayamalavagowla ragam notes
raga_notes = [sa, ri1, ga2, ma1, pa, da1, ni2, sam]
def play_raga(raga_notes):
  raga = music_pb2.NoteSequence()
  s_time = 0.0
# aarohana
  for swara in raga_notes:
    raga.notes.add(pitch=swara, start_time=s_time, end_time=s_time+0.5, velocity=80 )
    s_time += 0.5
  # avarohana
  for swara in raga_notes[::-1]:  # reverse notes and play as the number of notes are same as aarohana
    raga.notes.add(pitch=swara, start_time=s_time, end_time=s_time+0.5, velocity=80 )
    s_time += 0.5
  return raga 

# Example raga notes
# Mayamalavagowla ragam
mg = play_raga(raga_notes)
mg.total_time = 16
mg.tempos.add(qpm=60);
# This is a colab utility method that visualizes a NoteSequence.
note_seq.plot_sequence(mg)
# This is a colab utility method that plays a NoteSequence.
note_seq.play_sequence(mg,synth=note_seq.fluidsynth)
print("Mayamalavagowla [sa, ri1, ga2, ma1, pa, da1, ni2, sam] -[C,Db,E,F,G,Ab,B,C']")

# Shankarabharanam ragam
raga_notes = [sa, ri2, ga2, ma1, pa, da2, ni2, sam]
sb =play_raga(raga_notes)
sb.total_time = 16
sb.tempos.add(qpm=60);
note_seq.plot_sequence(sb)
note_seq.play_sequence(sb,synth=note_seq.fluidsynth)
print("Sankarabharanam [sa, ri2, ga2, ma1, pa, da2, ni2, sam] - [C,D,E,F,G,A,B,C']")

# Kalyani ragam
raga_notes = [sa, ri2, ga2, ma2, pa, da2, ni2, sam]
kl =play_raga(raga_notes)
kl.total_time = 16
kl.tempos.add(qpm=60);
note_seq.plot_sequence(kl)
note_seq.play_sequence(kl,synth=note_seq.fluidsynth)
print("Kalyani [sa, ri2, ga2, ma2, pa, da2, ni2, sam]-[C,D,E,F#,G,A,B,C']")